In [ ]:
!pip install javalang
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install -q transformers==3.5.0 fast-trees
! git clone -q https://github.com/microsoft/CodeXGLUE.git
import re
import javalang
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Copilot-Robustness/NonFullContext-Result.csv',index_col='index')

In [ ]:
def levenshtein(seq1, seq2):
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
                
    return (matrix[size_x - 1, size_y - 1])


def levenshtein_normalized(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return matrix[size_x - 1, size_y - 1]/max(size_x, size_y)

target = list(df['body'])
original = list(df['generatedResultOriginal'])
perturbedEvaluator = list(df['generatedResultEvaluator'])
perturbatedPivoting = list(df['generatedResultPivoting'])
perturbatedPegasus = list(df['generatedResultPegasus'])

resultOriginalMethods = list(df['mvnTestResultOriginal'])
resultEvaluator = list(df['mvnTestResultEvaluator'])
resultPivoting = list(df['mvnTestResultPivoting'])
resultPegasus = list(df['mvnTestResultPegasus'])

In [ ]:
originalJavaDoc = list(df['javaDocFirstSentence'])
perturbedJavaDocEvaluator = list(df['perturbed_eval_1'])
perturbedJavaDocPivoting = list(df['pivotingPerturbed'])
perturbedJavaDocPegasus = list(df['pegasusPerturbed'])


#Lev distance computation for code

levDistanceTargetOriginal = []
levDistanceTargetEvaluator = []
levDistanceTargetPivoting = []
levDistanceTargetPegasus = []
levDistanceTargetOriginalNorm = []
levDistanceTargetEvaluatorNorm = []
levDistanceTargetPivotingNorm = []
levDistanceTargetPegasusNorm = []

levDistanceOriginalEvaluator = []
levDistanceOriginalPivoting = []
levDistanceOriginalPegasus = []
levDistanceOriginalEvaluatorNorm = []
levDistanceOriginalPivotingNorm = []
levDistanceOriginalPegasusNorm = []

for (targetMethod, originalMethod, perturbedEvaluatorMethod, perturbatedPivotingMethod, perturbatedPegasusMethod, resultOriginalMethod, resultEvaluatorMethod, resultPivotingMethod, resultPegasusMethod) in tqdm(zip(target, original, perturbedEvaluator, perturbatedPivoting, perturbatedPegasus, resultOriginalMethods, resultEvaluator, resultPivoting, resultPegasus)):

  originalBodyTokens = []
  targetBodyTokens = []

  if originalMethod == 'Empty Method' or resultOriginalMethod == 'Not Valid' or resultOriginalMethod == 'Syntax Error':
      
      # levDistanceOriginalEvaluator.append('None')
      # levDistanceOriginalEvaluatorNorm.append('None')

      # levDistanceOriginalPivoting.append('None')
      # levDistanceOriginalPivotingNorm.append('None')
      
      # levDistanceOriginalPegasus.append('None')
      # levDistanceOriginalPegasusNorm.append('None')

      levDistanceTargetOriginal.append('None')
      levDistanceTargetOriginalNorm.append('None')
     
  
  else:
    originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
    targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

    lev = levenshtein(originalBodyTokens, targetBodyTokens)
    levDistanceTargetOriginal.append(lev)

    levNorm = levenshtein_normalized(originalBodyTokens, targetBodyTokens)
    levDistanceTargetOriginalNorm.append(levNorm)

  ##################################################################################################
  
  if perturbedEvaluatorMethod == 'Empty Method' or resultEvaluatorMethod == 'Not Valid' or resultEvaluatorMethod == 'Syntax Error':
    
    levDistanceOriginalEvaluator.append('None')
    levDistanceOriginalEvaluatorNorm.append('None')

    levDistanceTargetEvaluator.append('None')
    levDistanceTargetEvaluatorNorm.append('None')
  
  else:
    try:
      originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
      evaluatorBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(perturbedEvaluatorMethod))]
      targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

      lev = levenshtein(originalBodyTokens, evaluatorBodyTokens)
      levDistanceOriginalEvaluator.append(lev)

      levNorm = levenshtein_normalized(originalBodyTokens, evaluatorBodyTokens)
      levDistanceOriginalEvaluatorNorm.append(levNorm)

      lev = levenshtein(targetBodyTokens, evaluatorBodyTokens)
      levDistanceTargetEvaluator.append(lev)

      levNorm = levenshtein_normalized(targetBodyTokens, evaluatorBodyTokens)
      levDistanceTargetEvaluatorNorm.append(levNorm)

    except Exception:
      
      levDistanceOriginalEvaluator.append('None')
      levDistanceOriginalEvaluatorNorm.append('None')

      levDistanceTargetEvaluator.append('None')
      levDistanceTargetEvaluatorNorm.append('None')

  ##################################################################################################

  if perturbatedPivotingMethod == 'Empty Method' or resultPivotingMethod == 'Not Valid' or perturbatedPivotingMethod == 'Syntax Error':
      
      levDistanceOriginalPivoting.append('None')
      levDistanceOriginalPivotingNorm.append('None')

      levDistanceTargetPivoting.append('None')
      levDistanceTargetPivotingNorm.append('None')

  else:          
      try:
        originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
        pivotingBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(perturbatedPivotingMethod))]
        targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

        lev = levenshtein(originalBodyTokens, pivotingBodyTokens)
        levDistanceOriginalPivoting.append(lev)

        levNorm = levenshtein_normalized(originalBodyTokens, pivotingBodyTokens)
        levDistanceOriginalPivotingNorm.append(levNorm)

        lev = levenshtein(targetBodyTokens, pivotingBodyTokens)
        levDistanceTargetPivoting.append(lev)

        levNorm = levenshtein_normalized(targetBodyTokens, pivotingBodyTokens)
        levDistanceTargetPivotingNorm.append(levNorm)


      except Exception:

        levDistanceOriginalPivoting.append('None')
        levDistanceOriginalPivotingNorm.append('None')

        levDistanceTargetPivoting.append('None')
        levDistanceTargetPivotingNorm.append('None')

  ##################################################################################################
  if perturbatedPegasusMethod == 'Empty Method' or resultPegasusMethod == 'Not Valid' or perturbatedPegasusMethod == 'Syntax Error':
      
      levDistanceOriginalPegasus.append('None')
      levDistanceOriginalPegasusNorm.append('None')

      levDistanceTargetPegasus.append('None')
      levDistanceTargetPegasusNorm.append('None')
  
  else:
      try:
        originalBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(originalMethod))]
        pegasusBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(perturbatedPegasusMethod))]
        targetBodyTokens = [token.value for token in list(javalang.tokenizer.tokenize(targetMethod))]

        lev = levenshtein(originalBodyTokens, pegasusBodyTokens)
        levDistanceOriginalPegasus.append(lev)

        levNorm = levenshtein_normalized(originalBodyTokens, pegasusBodyTokens)
        levDistanceOriginalPegasusNorm.append(levNorm)

        lev = levenshtein(targetBodyTokens, pegasusBodyTokens)
        levDistanceTargetPegasus.append(lev)

        levNorm = levenshtein_normalized(targetBodyTokens, pegasusBodyTokens)
        levDistanceTargetPegasusNorm.append(levNorm)



      except Exception:
        
        levDistanceOriginalPegasus.append('None')
        levDistanceOriginalPegasusNorm.append('None')
        
        levDistanceTargetPegasus.append('None')
        levDistanceTargetPegasusNorm.append('None')
  ##################################################################################################


len(levDistanceOriginalEvaluator)

892it [04:46,  3.11it/s]


892

In [ ]:
#Lev distance computation for javadoc

levDistanceOriginalJavaDocEvaluator = []
levDistanceOriginalJavaDocPivoting = []
levDistanceOriginalJavaDocPegasus = []

levDistanceOriginalJavaDocEvaluatorNorm = []
levDistanceOriginalJavaDocPivotingNorm = []
levDistanceOriginalJavaDocPegasusNorm = []

for (javaDocOriginal, javaDocEvaluator, javaDocPivoting, javaDocPegasus) in tqdm(zip(originalJavaDoc, perturbedJavaDocEvaluator, perturbedJavaDocPivoting, perturbedJavaDocPegasus)):

    levDistanceOriginalJavaDocEvaluator.append(levenshtein(javaDocOriginal.split(), javaDocEvaluator.split()))
    levDistanceOriginalJavaDocEvaluatorNorm.append(levenshtein_normalized(javaDocOriginal.split(), javaDocEvaluator.split()))

    levDistanceOriginalJavaDocPivoting.append(levenshtein(javaDocOriginal.split(), javaDocPivoting.split()))
    levDistanceOriginalJavaDocPivotingNorm.append(levenshtein_normalized(javaDocOriginal.split(), javaDocPivoting.split()))
    
    levDistanceOriginalJavaDocPegasus.append(levenshtein(javaDocOriginal.split(), javaDocPegasus.split()))
    levDistanceOriginalJavaDocPegasusNorm.append(levenshtein_normalized(javaDocOriginal.split(), javaDocPegasus.split()))


892it [00:02, 368.32it/s]


In [ ]:
codeBleuAgainstOriginal = []
codeBleuAgainstEvaluator = []
codeBleuAgainstPivoting = []
codeBleuAgainstPegasus = []


for (targetMethod, originalMethod, perturbedEvaluatorMethod, perturbatedPivotingMethod, perturbatedPegasusMethod, resultOriginalMethod, resultEvaluatorMethod, resultPivotingMethod, resultPegasusMethod) in tqdm(zip(target, original, perturbedEvaluator, perturbatedPivoting, perturbatedPegasus, resultOriginalMethods, resultEvaluator, resultPivoting, resultPegasus)):

  if originalMethod == 'Empty Method' or resultOriginalMethod == 'Not Valid' or resultOriginalMethod == 'Syntax Error':
      codeBleuAgainstOriginal.append('None')
  else:
      
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          originalBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(originalMethod))])

          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          
          with open('prediction.txt','w') as f:
            f.write(originalBodyTokens)
          
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
        
          infoScore = res[0]
          codeBleuScore = res[1].split(':')[1].split(',')[0]
        
          codeBleuAgainstOriginal.append('{}'.format(codeBleuScore))
      
      except Exception:
        codeBleuAgainstOriginal.append('None')
        

  if perturbedEvaluatorMethod == 'Empty Method' or resultEvaluatorMethod == 'Not Valid' or resultEvaluatorMethod == 'Syntax Error':
      codeBleuAgainstEvaluator.append('None')
  else:
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          evaluatorBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(perturbedEvaluatorMethod))])

          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          with open('prediction.txt','w') as f:
            f.write(evaluatorBodyTokens)
          
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
      
          infoScore = res[0]

          codeBleuScore = res[1].split(':')[1].split(',')[0]

          codeBleuAgainstEvaluator.append('{}'.format(codeBleuScore))

      except Exception:
        codeBleuAgainstEvaluator.append('None')
        

  if perturbatedPivotingMethod == 'Empty Method' or resultPivotingMethod == 'Not Valid' or resultPivotingMethod == 'Syntax Error':
      codeBleuAgainstPivoting.append('None')
  else:
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          pivotingBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(perturbatedPivotingMethod))])

          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          with open('prediction.txt','w') as f:
            f.write(pivotingBodyTokens)
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
      
          infoScore = res[0]
          codeBleuScore = res[1].split(':')[1].split(',')[0]
      
          codeBleuAgainstPivoting.append('{}'.format(codeBleuScore))


      except Exception:
          codeBleuAgainstPivoting.append('None')
      
  if perturbatedPegasusMethod == 'Empty Method' or resultPegasusMethod == 'Not Valid' or resultPegasusMethod == 'Syntax Error':
      codeBleuAgainstPegasus.append('None')
  
  else:
      groundTruthBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(targetMethod))])
      
      try:
          pegasusBodyTokens = ' '.join([token.value for token in list(javalang.tokenizer.tokenize(perturbatedPegasusMethod))])
      
          with open('reference.txt','w') as f:
            f.write(groundTruthBodyTokens)
          with open('prediction.txt','w') as f:
            f.write(pegasusBodyTokens)
          res = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs /content/reference.txt --hyp /content/prediction.txt --lang java --params 0.25,0.25,0.25,0.25
          
          infoScore = res[0]
          codeBleuScore = res[1].split(':')[1].split(',')[0]
          codeBleuAgainstPegasus.append('{}'.format(codeBleuScore))
      
      except Exception:
          codeBleuAgainstPegasus.append('None')

df['CodeBleuOriginal'] = codeBleuAgainstOriginal
df['CodeBleuEvaluator'] = codeBleuAgainstEvaluator
df['CodeBleuPivoting'] = codeBleuAgainstPivoting
df['CodeBleuPegasus'] = codeBleuAgainstPegasus

df['levenshteinTarget-Original-Code'] =  levDistanceTargetOriginal
df['levenshteinTarget-Original-Code-Normalized'] =  levDistanceTargetOriginalNorm

df['levenshteinTarget-Evaluator-Code'] =  levDistanceTargetEvaluator
df['levenshteinTarget-Evaluator-Code-Normalized'] =  levDistanceTargetEvaluatorNorm

df['levenshteinTarget-Pivoting-Code'] =  levDistanceTargetPivoting
df['levenshteinTarget-Pivoting-Code-Normalized'] =  levDistanceTargetPivotingNorm

df['levenshteinTarget-Pegasus-Code'] =  levDistanceTargetPegasus
df['levenshteinTarget-Pegasus-Code-Normalized'] =  levDistanceTargetPegasusNorm

######################################################################


df['levenshteinOriginal-Evaluator-Code'] = levDistanceOriginalEvaluator
df['levenshteinOriginal-Evaluator-Code-Normalized'] = levDistanceOriginalEvaluatorNorm

df['levenshteinOriginal-Pivoting-Code'] = levDistanceOriginalPivoting
df['levenshteinOriginal-Pivoting-Code-Normalized'] = levDistanceOriginalPivotingNorm


df['levenshteinOriginal-Pegasus-Code'] = levDistanceOriginalPegasus
df['levenshteinOriginal-Pegasus-Code-Normalized'] = levDistanceOriginalPegasusNorm

######################################################################

df['levenshteinOriginal-Evaluator-JavaDoc'] = levDistanceOriginalJavaDocEvaluator
df['levenshteinOriginal-Evaluator-JavaDoc-Normalized'] = levDistanceOriginalJavaDocEvaluatorNorm

df['levenshteinOriginal-Pivoting-JavaDoc'] = levDistanceOriginalJavaDocPivoting
df['levenshteinOriginal-Pivoting-JavaDoc-Normalized'] = levDistanceOriginalJavaDocPivotingNorm

df['levenshteinOriginal-Pegasus-JavaDoc'] = levDistanceOriginalJavaDocPegasus
df['levenshteinOriginal-Pegasus-JavaDoc-Normalized'] = levDistanceOriginalJavaDocPegasusNorm

# df.to_csv('Non-FullContext-Result1.csv')

892it [12:04,  1.23it/s]


In [ ]:
df.head()

In [ ]:
df.to_csv('NonFullContext-With-Stats.csv')